# Workflow

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import sys

sys.path.append('../hydra_tod/')

from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u
from utils import Leg_poly_proj, view_samples
from flicker_model import sim_noise, flicker_cov


In [2]:
import pickle
from simulation import MultiTODSimulation
# Load the simulation data from a pickle file
with open('multi_tod_simulation_data.pkl', 'rb') as f:
    multi_tod_sim = pickle.load(f)

In [3]:
# Make all the individual variables available for backward compatibility
t_list = multi_tod_sim.t_list
theta_c_setting = multi_tod_sim.theta_c_setting
phi_c_setting = multi_tod_sim.phi_c_setting
theta_c_rising = multi_tod_sim.theta_c_rising
phi_c_rising = multi_tod_sim.phi_c_rising
bool_map_setting = multi_tod_sim.bool_map_setting
bool_map_rising = multi_tod_sim.bool_map_rising
integrated_beam_setting = multi_tod_sim.integrated_beam_setting
integrated_beam_rising = multi_tod_sim.integrated_beam_rising
full_bool_map = multi_tod_sim.full_bool_map
pixel_indices = multi_tod_sim.pixel_indices
integrated_beam = multi_tod_sim.integrated_beam
Tsky_operator_setting = multi_tod_sim.Tsky_operator_setting
Tsky_operator_rising = multi_tod_sim.Tsky_operator_rising
sky_params = multi_tod_sim.sky_params
ntime = multi_tod_sim.ntime
ndiode_proj = multi_tod_sim.ndiode_proj
nd_rec_operator = multi_tod_sim.nd_rec_operator
nd_rec_params = multi_tod_sim.nd_rec_params
gain_proj = multi_tod_sim.gain_proj
gain_params_setting = multi_tod_sim.gain_params_setting
gain_params_rising = multi_tod_sim.gain_params_rising
gains_setting = multi_tod_sim.gains_setting
gains_rising = multi_tod_sim.gains_rising
fc = multi_tod_sim.fc
logfc = multi_tod_sim.logfc
f0 = multi_tod_sim.f0
noise_setting = multi_tod_sim.noise_setting
noise_rising = multi_tod_sim.noise_rising
Tsys_setting = multi_tod_sim.Tsys_setting
Tsys_rising = multi_tod_sim.Tsys_rising
TOD_setting = multi_tod_sim.TOD_setting
TOD_rising = multi_tod_sim.TOD_rising
pixels_c_setting = multi_tod_sim.pixels_c_setting
bool_map_c_setting = multi_tod_sim.bool_map_c_setting
calibration_1_index = multi_tod_sim.calibration_1_index
calibration_5_indices = multi_tod_sim.calibration_5_indices

# Additional constants from the original code
T_ndiode = multi_tod_sim.T_ndiode
rec_params = multi_tod_sim.rec_params
dtime = multi_tod_sim.dtime
alpha = multi_tod_sim.alpha
logf0_list = [multi_tod_sim.logf0]
sigma_2 = multi_tod_sim.sigma_2

In [4]:
init_Tsys_params = np.hstack([sky_params, nd_rec_params, nd_rec_params])
init_noise_params = [-4.8, 2.2]

gain_prior_cov_inv = np.zeros(4)
gain_prior_cov_inv[0] = 2.51 # DC mode: Prior STD ~10%
gain_prior_mean = gain_params_setting

prior_std_frac = 0.1
Tsky_prior_cov_inv = 1. / (prior_std_frac * sky_params)**2
calibration_indices = calibration_5_indices
Tsky_prior_cov_inv[calibration_indices] = 1e20

aux = np.ones_like(nd_rec_params)*0.0
aux[0] = 1
# Trec_prior_cov_inv = [aux] # Effectlively no prior

Trec_prior_cov_inv_list = [aux, aux]


def log_prior_noise(params):
    _, alpha_est = params
    return -1e10 * (alpha_est - alpha)**4 

In [5]:
# Import the FlickerCorrEmulator class first
from flicker_model import FlickerCorrEmulator
from full_Gibbs_sampler import full_Gibbs_sampler_multi_TODS_v2

Tsys_samples, all_gain_samples, all_noise_samples = \
    full_Gibbs_sampler_multi_TODS_v2([TOD_setting, TOD_rising],
                                    [t_list, t_list],
                                    [gain_proj, gain_proj],
                                    [Tsky_operator_setting, Tsky_operator_rising],
                                    [nd_rec_operator, nd_rec_operator],
                                    init_Tsys_params,
                                    init_noise_params, 
                                    [logfc, logfc],
                                    num_Jeffrey=False,
                                    wnoise_var=2.5e-6,
                                    Tsky_prior_cov_inv=Tsky_prior_cov_inv,
                                    Tsky_prior_mean=sky_params,
                                    local_Trec_prior_cov_inv_list=Trec_prior_cov_inv_list,
                                    local_Trec_prior_mean_list=[nd_rec_params, nd_rec_params],
                                    local_gain_prior_cov_inv_list=[gain_prior_cov_inv,gain_prior_cov_inv],
                                    local_gain_prior_mean_list=[gain_params_setting,gain_params_rising],
                                    # local_noise_prior_func_list=[log_prior_noise, log_prior_noise],
                                    n_samples=1000,
                                    tol=1e-12)

Using the emulator for flicker noise correlation function.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 0: [6.31184639 0.41936058 0.26553246 0.05866948]
Rank: 0, local id: 0, noise_sample 0: [-5.09101289  1.98258455]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 0: [6.8452648  0.14232119 0.7441142  0.78390756]
Rank: 0, local id: 1, noise_sample 0: [-5.12567206  1.7631733 ]
Reached max iterations with fractional norm error 0.001104723190129885.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 1: [6.31224122 0.42093069 0.26489823 0.05659098]
Rank: 0, local id: 0, noise_sample 1: [-4.5361641   2.28107443]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 1: [6.84516899 0.14236166 0.74565522 0.78112098]
Rank: 0, local id: 1, noise_sample 1: [-4.7582574  1.9561421]
Reached max iterations with fractional norm error 0.00799084557263284.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 2: [6.31218667 0.41913062 0.26621314 0.0595

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 0, noise_sample 3: [-5.00560154  1.89889165]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 3: [6.84490092 0.14253042 0.74679856 0.77799743]
Rank: 0, local id: 1, noise_sample 3: [-5.98137087  3.64241482]
Reached max iterations with fractional norm error 0.00028988310963103743.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 4: [6.31236866 0.41735082 0.2646337  0.06222156]
Rank: 0, local id: 0, noise_sample 4: [-4.46022361  2.6442083 ]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 4: [6.844133   0.14219503 0.74602451 0.77721792]
Rank: 0, local id: 1, noise_sample 4: [-4.11546707  3.80943967]
Reached max iterations with fractional norm error 0.000544812708887286.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 5: [6.31300595 0.41752537 0.26456693 0.06280891]
Rank: 0, local id: 0, noise_sample 5: [-5.7417026   2.77094409]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 5: [6.84437264 0.1412851  0.74542

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 1, noise_sample 10: [-5.21629124  2.55983223]
Reached max iterations with fractional norm error 0.0010571516981289348.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 11: [6.31313469 0.4174657  0.26429977 0.06315846]


...

Rank: 0, local id: 0, noise_sample 995: [-3.08055183  3.97065015]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 995: [6.80961037 0.13726768 0.75712126 0.77532575]
Rank: 0, local id: 1, noise_sample 995: [-3.76681911  2.50175808]
Reached max iterations with fractional norm error 0.00021317383265842272.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 996: [6.29244939 0.41829505 0.2781985  0.097679  ]
Rank: 0, local id: 0, noise_sample 996: [-3.22259899  3.22644454]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 996: [6.8087421  0.14354699 0.73987161 0.75234256]
Rank: 0, local id: 1, noise_sample 996: [-4.84390515  1.6389666 ]
Reached max iterations with fractional norm error 0.0007358817289753788.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 997: [6.29129583 0.39805006 0.25803608 0.0921511 ]
Rank: 0, local id: 0, noise_sample 997: [-4.13534732  1.84702639]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 997: [6.808239

/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)
/opt/homebrew/Caskroom/miniconda/base/envs/TOD/lib/python3.13/s

Rank: 0, local id: 1, noise_sample 998: [-3.92690286  2.87900789]
Reached max iterations with fractional norm error 0.001237308208068745.
Converged in 5 iterations.
Rank: 0, local id: 0, gain_sample 999: [6.29114826 0.41515534 0.25129947 0.06812035]


/Users/zzhang/Workspace/flicker/notebooks/../hydra_tod/noise_sampler_fixed_fc.py:50: RuntimeWarning: invalid value encountered in log
  result=n * np.log(a0) + np.dot(factors, terms)


Rank: 0, local id: 0, noise_sample 999: [-3.1615982   3.83868599]
Converged in 5 iterations.
Rank: 0, local id: 1, gain_sample 999: [6.80837692 0.13733563 0.73714886 0.77191293]
Rank: 0, local id: 1, noise_sample 999: [-4.58729465  2.07135521]
Reached max iterations with fractional norm error 0.004066922084640064.


In [6]:
# First, save the samples as npy files..

np.save("outputs/GS5_db/Tsys_samples.npy", Tsys_samples)
np.save("outputs/GS5_db/gain_samples.npy", np.concatenate(all_gain_samples, axis=0))
np.save("outputs/GS5_db/noise_samples.npy", np.concatenate(all_noise_samples, axis=0))